# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\gwmch\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [7]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Write your code below.
import warnings 
warnings.filterwarnings('ignore') # mute warning messages

#dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# provided by Julia Ren, the npartitions parameter allow parallel processing
dd_px = dd.read_parquet(parquet_files, npartitions=len("ticker")).set_index("ticker")


In [21]:
import numpy as np
dd_feats = (dd_px.groupby('ticker', group_keys=False).apply(
        lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
        returns = lambda x: x['Close']/x['Close_lag_1'] -1
).assign(
        hi_lo_range = lambda x:x['High']-x['Low']
)
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [23]:
# Write your code below.
df_feats = dd_feats.compute()

In [25]:
df_feats2 = df_feats.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg = x['returns'].rolling(10).mean())
)


In [26]:
df_feats2.head(20)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,hi_lo_range,rolling_avg
ticker,,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.710287,140.101950,137.431261,138.268341,1441600,0.0,0.0,2024,NaN,NaN,2.670689,NaN
A,2024-01-03 00:00:00-05:00,137.520949,137.520949,130.615013,130.704697,2074500,0.0,0.0,2024,138.268341,-0.054703,6.905936,NaN
A,2024-01-04 00:00:00-05:00,130.096807,131.043507,129.738057,130.545242,2446600,0.0,0.0,2024,130.704697,-0.001220,1.305450,NaN
A,2024-01-05 00:00:00-05:00,129.548727,131.501930,128.173513,130.106781,1394000,0.0,0.0,2024,130.545242,-0.003359,3.328417,NaN
A,2024-01-08 00:00:00-05:00,129.688234,133.106335,129.359378,132.916992,1311400,0.0,0.0,2024,130.106781,0.021599,3.746957,NaN
A,2024-01-09 00:00:00-05:00,131.810841,135.179097,129.558677,130.226349,1435000,0.0,0.0,2024,132.916992,-0.020243,5.620421,NaN
A,2024-01-10 00:00:00-05:00,130.126709,130.704698,127.914415,130.634933,1326300,0.0,0.0,2024,130.226349,0.003137,2.790283,NaN
A,2024-01-11 00:00:00-05:00,130.126713,130.226357,127.456016,129.229828,2060500,0.0,0.0,2024,130.634933,-0.010756,2.770341,NaN
A,2024-01-12 00:00:00-05:00,129.857640,131.153131,129.189968,130.086838,1285200,0.0,0.0,2024,129.229828,0.006632,1.963163,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it's not necessary.

It would've been better to do it in Dask because Dask can do calculations in parallel, which will speed up the calculations.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.